In [2]:
using Pkg
Pkg.instantiate()
using MCMRSimulator
using CairoMakie
using Statistics
using DelimitedFiles
using FileIO
using Printf
using JLD2


In [2]:
using Random

function repelling_random_position(radius, pos_in, repeatsize; maxiter=1000, repulsion_strength=0.01)
    pos = deepcopy(pos_in)
    size = repeatsize
    # Initialize circles list
    
    function does_repulsed_have_a_collision(circle, index)
        if circle[1] > size/2 || circle[1] < -size/2
            return true
        elseif circle[2] > size/2 || circle[2] < -size/2
            return true
        end
        for i in 1:length(pos)
            if i != index
                other_circle = pos[i]
                a = 2*radius
                x = mod((circle[1] - other_circle[1] + size/2), size) - size/2
                y = mod((circle[2] - other_circle[2] + size/2), size) - size/2

                if a >= sqrt(x^2 + y^2)
                    return true
                end
            end
        end

        return false
    end
    

        # Update circle positions based on repulsion
    for a in 1:maxiter
        success_count=0
        for i in 1:length(pos)
            start_pos = pos[i]
            proposed_pos = pos[i]
            for j in 1:length(pos)
                if i != j
                    # ((abs(circle[1][1] - other_circle[1][1]) + size/2) % size) - size/2
                    dx = mod((pos[j][1] - pos[i][1] + size/2), size) - size/2
                    dy =  mod((pos[j][2] - pos[i][2] + size/2), size) - size/2
                    dist = sqrt(dx^2 + dy^2)
                    # force = (radius^2*radius^2)*repulsion_strength / (dist * mean_radius^2)
                    force = (radius^2)*repulsion_strength / (dist)
                    proposed_pos = (proposed_pos[1] - force * dx/dist, proposed_pos[2] - force * dy/dist)
                    # println(dx, dy)
                end
            end
            # Abandon changes if there's overlap
            if does_repulsed_have_a_collision(proposed_pos, i)
                pos[i] = start_pos
            else
                pos[i] = proposed_pos
                success_count += 1
            end
        end
        if success_count == 0
            println(a)
            break
        end
        # println(a)
    end
    # rng_pos = []
    # for i in 1:length(circles)
    #     push!(rng_pos, Float64.(circles[i][1]))
    # end
    # bounding_box = MCMRSimulator.BoundingBox([-size/2, -size/2, -500], [size/2, size/2, 500])
        
    return pos
end

repelling_random_position (generic function with 1 method)

In [3]:
T2s = 10:10:300
T2_bound=1e-3
rho=0.65
r0 = 0.1
rep = r0*100
res = MCMRSimulator.random_positions_radii([rep*1.001, rep*1.001], rho*1.001^2, 2, mean=r0*1.001, variance=0)
repeled = repelling_random_position(r0*1.001, res[1], rep*1.001, maxiter=1000, repulsion_strength=3e-3)
t_dwell = 30
for r in 0.1:0.1:7.5
    rep = r*100
    # rng_geom = Cylinders(position=repeled.*(r/r0), radius=r, repeats=[rep, rep])
    svratio = 2/r
    for T2 = T2s # ms
        surf_dens = 1/svratio*t_dwell/T2
        rng_geom = Cylinders(position=repeled.*(r/r0), radius=r, dwell_time=t_dwell, density=surf_dens, repeats=[rep, rep], R2_surface=1/T2_bound)
        MCMRSimulator.write_geometry("./cylinders_rep100r/MT/cylinders_MT_"* string(T2) *"_sus_0_perm_0.000_rmean_"*@sprintf("%.2f",r)*"_density_0.65.json", rng_geom)
    end
    for perm in 0:0.001:0.02
        rng_geom = Cylinders(position=repeled.*(r/r0), radius=r, permeability=perm, repeats=[rep, rep])
        MCMRSimulator.write_geometry("./cylinders_rep100r/permeability/cylinders_MT_0_sus_0_perm_"*@sprintf("%.3f",perm)*"_rmean_"*@sprintf("%.2f",r)*"_density_0.65.json", rng_geom)
    end
    # MCMRSimulator.write_geometry("cylinders_rmean_"*@sprintf("%.2f",r)*"_density_"*@sprintf("%.2f",rho)*".json", rng_geom)
end


There are 2074 Cylinder based on values for Field(position)
There are 2074 Cylinder based on values for Field(position)
There are 2074 Cylinder based on values for Field(position)
There are 2074 Cylinder based on values for Field(position)
There are 2074 Cylinder based on values for Field(position)
There are 2074 Cylinder based on values for Field(position)
There are 2074 Cylinder based on values for Field(position)
There are 2074 Cylinder based on values for Field(position)
There are 2074 Cylinder based on values for Field(position)
There are 2074 Cylinder based on values for Field(position)
There are 2074 Cylinder based on values for Field(position)
There are 2074 Cylinder based on values for Field(position)
There are 2074 Cylinder based on values for Field(position)
There are 2074 Cylinder based on values for Field(position)
There are 2074 Cylinder based on values for Field(position)
There are 2074 Cylinder based on values for Field(position)
There are 2074 Cylinder based on values 

## Test bounding box

In [3]:
seq = dwi(qval=0.01)

Sequence (TR=80.0ms):
    - InstantRFPulse: t=0.0ms, θ=90.0°, ϕ=-90.0°;
    - InstantRFPulse: t=40.0ms, θ=180.0°, ϕ=0.0°;
    - Readout at 80.0ms


In [4]:
sim = Simulation(seq, diffusivity=2)

Simulation(Geometry(), D=2.0um^2/ms, GlobalProperties()):
1 sequences:
Sequence (TR=80.0ms):
    - InstantRFPulse: t=0.0ms, θ=90.0°, ϕ=-90.0°;
    - InstantRFPulse: t=40.0ms, θ=180.0°, ϕ=0.0°;
    - Readout at 80.0ms


In [5]:
s = readout(1000, sim, bounding_box=BoundingBox([-10,-10,-10],[10,10,10]))

SpinOrientationSum(longitudinal=-1.0835776720341528e-13, transverse=996.2219485805227, phase=-0.014247930398482822°, nspins=1000)